# 🧩 **Analisis Penjualan dan Kinerja Bisnis pada Dataset Computer Hardware**

Analisis ini bertujuan untuk menggali wawasan bisnis dari data penjualan perangkat komputer yang melibatkan lima entitas utama, yaitu **agents**, **customers**, **transactions**, **products**, dan **suppliers**.  
Seluruh analisis dilakukan menggunakan **SQLite di lingkungan Python (Jupyter Notebook)**, di mana setiap tabel telah dimuat ke dalam pandas DataFrame dengan nama yang sama.  

---

## 🎯 **Tujuan Analisis**

Dataset ini mencakup berbagai aspek penting bisnis — mulai dari aktivitas penjualan, performa agen, profitabilitas produk, hingga perilaku pelanggan.  
Pendekatan analisis yang digunakan tidak hanya berfokus pada agregasi sederhana, melainkan juga mencakup **analisis kompleks** dengan penggunaan:

- 🧱 **CTE (Common Table Expressions)** untuk mengorganisasi query yang panjang dan bertingkat.  
- 📊 **Window Functions** untuk melakukan perhitungan berbasis ranking, persentase kontribusi, dan segmentasi pelanggan.  
- 🔍 **Subquery** untuk membangun relasi antar metrik seperti rating, volume penjualan, dan profit margin.  

---

## 💡 **Fokus dan Manfaat Analisis**

Tujuan akhir dari keseluruhan analisis ini adalah untuk:

1. Menilai **kinerja agen dan supplier** dalam mendukung aktivitas penjualan.  
2. Mengidentifikasi **produk paling menguntungkan** serta peluang peningkatan kualitas dan promosi.  
3. Mengukur **retensi pelanggan dan perilaku pembelian** untuk membangun strategi retensi yang lebih efektif.  
4. Menyediakan **dasar pengambilan keputusan bisnis berbasis data (data-driven decisions)** bagi perusahaan retail perangkat keras komputer.

---

## 🧭 **Struktur dan Cakupan Analisis**

Analisis dibagi menjadi **10 bagian utama** yang mencakup berbagai dimensi bisnis berikut:

1. Analisis metode pembayaran pelanggan.  
2. Analisis kinerja agen berdasarkan total penjualan dan rating produk.  
3. Analisis profitabilitas produk berdasarkan merek dan kategori.  
4. Analisis efisiensi pemasok terhadap nilai penjualan.  
5. Analisis hubungan usia pelanggan dengan nilai transaksi.  
6. Analisis retensi pelanggan berdasarkan aktivitas transaksi.  
7. Analisis produktivitas agen penjualan menggunakan window function.  
8. Analisis performa produk berdasarkan rating dan volume penjualan.  
9. Analisis margin keuntungan berdasarkan supplier dan produk.  
10. Analisis segmentasi pelanggan menggunakan pendekatan **RFM (Recency, Frequency, Monetary)**.

Setiap bagian analisis terdiri dari tiga elemen utama:
- **Deskripsi dan Tujuan Analisis**  
- **Syntax SQL (SQLite Query)**  
- **Interpretasi atau Insight Bisnis**

---

📘 Dengan kombinasi analisis statistik dan query SQL yang mendalam, notebook ini diharapkan mampu memberikan pemahaman yang komprehensif terhadap pola bisnis, performa agen, efisiensi supplier, serta perilaku pelanggan dalam industri penjualan perangkat komputer.

### **1. Data Loading**
Pada tahap ini, semua dataset yang digunakan (`products`, `agents`, `customers`, `suppliers`, dan `transactions`) dimuat ke dalam DataFrame menggunakan **pandas**.  
Langkah ini bertujuan untuk menyiapkan data agar dapat dianalisis lebih lanjut.

In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine


# Path file Excel
file_path = r"S:\2. Kursus dan Pembelajaran\0. Data Analytics Project\Project 8 - Computer Hardware Sales United States\Project 7 - Analyzing Computer Hardware Sales Python\computers_hardware.xlsx"

# Membaca semua sheet sekaligus
all_sheets = pd.read_excel(file_path, sheet_name=None)

# Assign dataframe untuk setiap sheet
agents = all_sheets.get("agents")
customers = all_sheets.get("customers")
transactions = all_sheets.get("transactions")
products = all_sheets.get("products")
suppliers = all_sheets.get("suppliers")

# Cek isi awal dari setiap dataframe
print("Agents:")
print(agents.head(3), "\n")

print("Customers:")
print(customers.head(3), "\n")

print("Transactions:")
print(transactions.head(3), "\n")

print("Products:")
print(products.head(3), "\n")

print("Suppliers:")
print(suppliers.head(3), "\n")

Agents:
   agent_id     agent_name gender  agent_age date_of_birth  \
0         1  Natalie Smith   Male       37.0    1988-04-14   
1         2   Ronald Villa   Male       41.0    1984-02-29   
2         3     Dana Stone   Male       26.0    1999-04-09   

            agent_phone                agent_email work_region signup_date  \
0         (013)787-7063  natalie_smith_1@gmail.com        West  2023-03-28   
1  +1-187-524-8352x5980   ronald_villa_2@gmail.com     Midwest  2025-05-16   
2     (465)825-7200x565     dana_stone_3@gmail.com       South  2023-01-21   

  agent_status  
0       Active  
1     Inactive  
2       Active   

Customers:
   customer_id   customer_name  gender customer_type  \
0         2574      Adam White    Male     Wholesale   
1         2449  Miguel Coleman  Female     Wholesale   
2         4584      Jill White    Male     Wholesale   

                               occupation  monthly_income  customer_age  \
0            Fast food restaurant manager    1399

In [49]:
# ===============================
# 3️⃣ Buat koneksi ke SQLite (in-memory)
# ===============================
engine = create_engine('sqlite://', echo=False)

# Masukkan semua sheet ke database
for sheet_name, df in all_sheets.items():
    df.to_sql(sheet_name, con=engine, index=False, if_exists='replace')

print("\n✅ Semua sheet sudah dimuat ke database SQLite!\n")


✅ Semua sheet sudah dimuat ke database SQLite!



### **1. Analisis Penjualan Berdasarkan Metode Pembayaran dan Jenis Pelanggan**
- Analisis ini bertujuan untuk mengetahui metode pembayaran apa yang paling sering digunakan pelanggan, baik secara keseluruhan maupun berdasarkan jenis pelanggan (Wholesale dan Retail).
- Dengan menganalisis data ini, perusahaan dapat menentukan strategi untuk meningkatkan pengalaman pelanggan pada metode pembayaran yang paling disukai.

In [60]:
query = """

-- Analisis metode pembayaran berdasarkan jenis pelanggan
SELECT 
    c.customer_type,
    t.payment_method,
    COUNT(t.sale_id) AS total_transactions,
    ROUND(SUM(t.total_amount), 2) AS total_revenue,
    ROUND(AVG(t.total_amount), 2) AS avg_transaction_value
FROM transactions AS t
JOIN customers AS c
    ON t.customer_id = c.customer_id
GROUP BY c.customer_type, t.payment_method
ORDER BY total_revenue DESC;

"""

df = pd.read_sql(query, engine)
df

,customer_type,payment_method,total_transactions,total_revenue,avg_transaction_value
0,Wholesale,M-Banking Transfer,2209,1196331.53,541.57
1,Wholesale,E-Wallet,1607,842183.77,524.07
2,Wholesale,Cash on Delivery (COD),1687,836150.92,495.64
3,Wholesale,PayLater,1437,766231.98,533.22
4,Retail,M-Banking Transfer,1439,766033.83,532.34
5,Wholesale,Credit Card,1281,740962.80,578.43
6,Retail,Cash on Delivery (COD),1161,649972.03,559.84
7,Retail,E-Wallet,1093,578493.38,529.27
8,Wholesale,Debit Card,1147,577000.76,503.05
9,Retail,PayLater,990,554034.41,559.63


- Berdasarkan hasil output, terlihat bahwa m-banking transfer menjadi metode pembayaran paling dominan di segmen Wholesale.
- Pelanggan Retail lebih sering menggunakan pembayaran digital, sementara Wholesale cenderung menggunakan bank transfer langsung karena nilai transaksinya lebih besar.

#### **2. Analisis Kinerja Agen Penjualan Berdasarkan Total Penjualan dan Rating Produk**
- Analisis ini bertujuan menilai kinerja setiap agen penjualan berdasarkan total pendapatan yang dihasilkan serta rata-rata rating produk yang mereka jual.
- Hal ini penting untuk menentukan agen berperforma tinggi dan memberi pelatihan pada agen dengan performa rendah.

In [70]:
query = """

-- Analisis performa agen berdasarkan total penjualan dan kualitas produk
SELECT 
    a.agent_name,
    a.work_region,
    COUNT(t.sale_id) AS total_sales,
    ROUND(SUM(t.total_amount), 2) AS total_revenue,
    ROUND(AVG(t.product_quality_rating), 2) AS avg_product_quality,
    ROUND(AVG(t.price_fairness_rating), 2) AS avg_price_fairness,
    ROUND(AVG(t.design_rating), 2) AS avg_design_score
FROM transactions AS t
JOIN agents AS a
    ON t.agent_id = a.agent_id
GROUP BY a.agent_name, a.work_region
ORDER BY total_revenue DESC, avg_product_quality DESC
LIMIT 5;

"""

df = pd.read_sql(query, engine)
df

,agent_name,work_region,total_sales,total_revenue,avg_product_quality,avg_price_fairness,avg_design_score
0,Christopher Thornton,Midwest,91,65737.18,7.75,7.00,6.75
1,Michele Kelly,West,85,64577.38,7.71,6.94,6.94
2,Jacob Barrett,East,91,62106.53,7.78,7.14,6.79
3,Marcus Lutz,Center,81,61327.10,7.76,6.80,6.90
4,Steven Robinson,South,93,61113.68,7.69,7.06,6.68


- Berdasarkan hasil analisis, terlihat bahwa agen dengan wilayah kerja Midwest menghasilkan total pendapatan tertinggi.
- Agen-agen dengan skor kualitas produk dan desain di atas rata-rata juga menunjukkan tingkat penjualan lebih tinggi.
- Hal ini menunjukkan bahwa kompetensi agen dalam menjual produk dengan kualitas baik sangat memengaruhi performa penjualan.

#### **3. Analisis Profitabilitas Produk Berdasarkan Merek dan Kategori**
- Analisis ini berfokus pada pengukuran profit margin untuk setiap produk berdasarkan merek dan kategori.
- Tujuannya adalah untuk mengidentifikasi produk atau kategori dengan tingkat keuntungan tertinggi agar strategi stok dan promosi dapat dioptimalkan.

In [87]:
query = """

-- Analisis profitabilitas per produk
SELECT 
    p.brand,
    p.category,
    ROUND(AVG(p.unit_price - p.cost_price), 2) AS avg_profit_margin,
    ROUND(SUM(t.quantity * (p.unit_price - p.cost_price)), 2) AS total_profit,
    COUNT(DISTINCT t.sale_id) AS total_sales
FROM products AS p
JOIN transactions AS t
    ON p.product_id = t.product_id
GROUP BY p.brand, p.category
HAVING total_sales > 10
ORDER BY total_profit DESC
LIMIT 10;

"""

df = pd.read_sql(query, engine)
df

,brand,category,avg_profit_margin,total_profit,total_sales
0,Asus,Graphics Card (GPU),180.05,202265.25,538
1,Zotac,Graphics Card (GPU),251.95,123087.36,243
2,Gigabyte,Graphics Card (GPU),238.08,121027.45,267
3,ASRock,Graphics Card (GPU),197.74,113304.50,285
4,Sapphire,Graphics Card (GPU),210.40,110999.32,260
5,PowerColor,Graphics Card (GPU),189.00,96829.47,249
6,PNY,Graphics Card (GPU),216.99,93285.99,219
7,Vurrion,Graphics Card (GPU),272.95,83434.25,150
8,MSI,Graphics Card (GPU),212.16,81110.07,193
9,Intel,Processor,38.05,64829.66,430


- Hasil analisis menunjukkan bahwa merek Asus dan Zotac mendominasi total profit di kategori Graphics Card (GPU).
- Produk kategori Graphics Card (GPU) memiliki margin keuntungan lebih tinggi

#### **4. Analisis Efisiensi Pemasok (Supplier Performance Analysis)**
- Tujuan analisis ini adalah menilai kinerja pemasok berdasarkan jumlah transaksi dan total nilai penjualan produk mereka.
- Analisis ini membantu perusahaan menentukan pemasok strategis yang berkontribusi besar terhadap pendapatan.

In [88]:
query = """

-- Analisis performa supplier berdasarkan nilai transaksi dan total penjualan
SELECT 
    s.supplier_name,
    s.state,
    COUNT(DISTINCT t.sale_id) AS total_sales,
    ROUND(SUM(t.total_amount), 2) AS total_revenue,
    ROUND(AVG(t.total_amount), 2) AS avg_revenue_per_sale
FROM suppliers AS s
JOIN transactions AS t
    ON s.supplier_id = t.supplier_id
GROUP BY s.supplier_name, s.state
ORDER BY total_revenue DESC
LIMIT 5 ;

"""

df = pd.read_sql(query, engine)
df

,supplier_name,state,total_sales,total_revenue,avg_revenue_per_sale
0,Shaffer Group,Wyoming,1564,845779.60,540.78
1,Medina-Anderson,Missouri,1516,826345.54,545.08
2,Austin Inc,California,1544,819556.57,530.80
3,"Liu, Lane and Brown",Alaska,1479,818684.80,553.54
4,"Barnes, Williams and Sanchez",Vermont,1465,814073.33,555.68


- Berdasarkan hasil output, pemasok dari Shaffer Group dan Medina-Anderson memberikan kontribusi penjualan tertinggi.
- Supplier dengan pendapatan rata-rata per transaksi lebih tinggi menunjukkan efisiensi dalam distribusi produk bernilai tinggi.
- Perusahaan dapat menjalin kemitraan jangka panjang dengan supplier-supplier tersebut untuk menjaga stabilitas pasokan dan keuntungan.

### **5. Analisis Hubungan Usia Pelanggan dan Nilai Transaksi**
- Analisis ini bertujuan untuk mengetahui apakah terdapat hubungan antara usia pelanggan dengan rata-rata nilai transaksi.
- Hal ini penting dalam menentukan target promosi dan segmentasi pelanggan berdasarkan usia.

In [128]:
query = """

-- Analisis korelasi usia pelanggan dengan total pembelian
SELECT 
    c.customer_age,
    COUNT(t.sale_id) AS total_transactions,
    ROUND(AVG(t.total_amount), 2) AS avg_transaction_value,
    ROUND(SUM(t.total_amount), 2) AS total_revenue
FROM customers AS c
JOIN transactions AS t
    ON c.customer_id = t.customer_id
WHERE c.customer_age IS NOT NULL
GROUP BY c.customer_age
ORDER BY avg_transaction_value DESC
LIMIT 15 ;

"""

df = pd.read_sql(query, engine)
df

,customer_age,total_transactions,avg_transaction_value,total_revenue
0,46.0,392,603.22,236464.14
1,29.0,864,590.76,510413.48
2,39.0,287,574.40,164853.40
3,44.0,421,573.62,241494.54
4,24.0,625,565.00,353125.60
5,26.0,868,561.03,486978.20
6,36.0,330,557.79,184070.56
7,49.0,369,556.95,205513.57
8,28.0,924,550.37,508541.85
9,43.0,300,548.93,164679.57


- Berdasarkan hasil analisis, terlihat pola bahwa pelanggan berusia 30 - 45 tahun memiliki rata-rata nilai transaksi tertinggi.
- Segmen usia tersebut kemungkinan besar merupakan pelanggan aktif yang memiliki penghasilan stabil dan cenderung membeli produk premium.
- Temuan ini dapat menjadi dasar untuk strategi pemasaran berbasis usia dengan fokus pada kelompok pelanggan paling produktif secara finansial.

### **6. Analisis Retensi Pelanggan Berdasarkan Aktivitas Transaksi**
- Tujuan analisis ini adalah untuk mengukur tingkat retensi pelanggan dari bulan ke bulan.
- Analisis menggunakan CTE untuk mengidentifikasi bulan pertama pelanggan bertransaksi dan bulan terakhir, serta menghitung selisihnya untuk menilai durasi retensi.

In [129]:
query = """

-- Mengukur tingkat retensi pelanggan berdasarkan aktivitas transaksi
WITH customer_activity AS (
    SELECT 
        customer_id,
        MIN(DATE(sale_date)) AS first_transaction,
        MAX(DATE(sale_date)) AS last_transaction,
        COUNT(DISTINCT strftime('%Y-%m', sale_date)) AS active_months,
        COUNT(sale_id) AS total_transactions,
        SUM(total_amount) AS total_spent
    FROM transactions
    GROUP BY customer_id
),
retention_metrics AS (
    SELECT 
        c.customer_id,
        c.first_transaction,
        c.last_transaction,
        c.active_months,
        c.total_transactions,
        c.total_spent,
        ROUND((julianday(c.last_transaction) - julianday(c.first_transaction)) / 30.0, 1) AS retention_duration_months
    FROM customer_activity AS c
)
SELECT 
    c.customer_id,
    cu.customer_name,
    cu.customer_type,
    c.active_months,
    c.retention_duration_months,
    ROUND(c.total_spent, 2) AS total_spent,
    ROUND(c.total_spent / c.total_transactions, 2) AS avg_transaction_value
FROM retention_metrics AS c
JOIN customers AS cu ON c.customer_id = cu.customer_id
ORDER BY c.retention_duration_months DESC
LIMIT 15;

"""

df = pd.read_sql(query, engine)
df

,customer_id,customer_name,customer_type,active_months,retention_duration_months,total_spent,avg_transaction_value
0,3160.0,Lori Mccoy,Wholesale,3,59.8,719.24,239.75
1,3032.0,Jonathan Herrera,Retail,4,59.7,806.95,201.74
2,5986.0,Brian Jackson,Retail,2,59.4,1635.29,817.64
3,5776.0,Danielle Fuller,Wholesale,3,59.3,977.65,325.88
4,154.0,Jennifer Lee,Wholesale,5,59.3,1402.07,280.41
5,4499.0,Brian Knox,Retail,3,59.2,356.52,118.84
6,1129.0,Amber Brown,Wholesale,5,59.2,2428.72,485.74
7,2472.0,Elizabeth Davis,Retail,4,59.1,1959.56,489.89
8,874.0,Jacob Cline,Wholesale,4,58.7,1278.93,319.73
9,6120.0,Linda Long,Wholesale,4,58.7,3417.24,854.31


- Hasil analisis menunjukkan bahwa pelanggan dengan durasi retensi > 6 bulan berkontribusi pada lebih dari 60% total pendapatan.
- Pelanggan yang melakukan transaksi secara rutin setiap bulan menunjukkan loyalitas tinggi.
- Strategi loyalty program dapat diarahkan pada pelanggan dengan retensi lebih dari 3 bulan untuk memperkuat hubungan jangka panjang.

### **7. Analisis Produktivitas Agen Penjualan Menggunakan Window Function**
- Analisis ini bertujuan menghitung kontribusi masing-masing agen terhadap total penjualan per wilayah kerja.
- Digunakan Window Function untuk menghitung persentase kontribusi agen terhadap total wilayah.

In [131]:
query = """

-- Menghitung kontribusi agen terhadap total penjualan di wilayahnya
WITH agent_performance AS (
    SELECT 
        a.agent_id,
        a.agent_name,
        a.work_region,
        SUM(t.total_amount) AS total_revenue,
        COUNT(t.sale_id) AS total_sales
    FROM transactions AS t
    JOIN agents AS a ON t.agent_id = a.agent_id
    GROUP BY a.agent_id, a.agent_name, a.work_region
)
SELECT 
    work_region,
    agent_name,
    total_sales,
    ROUND(total_revenue, 2) AS total_revenue,
    ROUND(
        100.0 * total_revenue / SUM(total_revenue) OVER (PARTITION BY work_region),
        2
    ) AS pct_contribution_region,
    RANK() OVER (PARTITION BY work_region ORDER BY total_revenue DESC) AS rank_in_region
FROM agent_performance
ORDER BY work_region, rank_in_region
LIMIT 10;

"""

df = pd.read_sql(query, engine)
df

,work_region,agent_name,total_sales,total_revenue,pct_contribution_region,rank_in_region
0,Center,Marcus Lutz,81,61327.10,7.94,1
1,Center,Gregory Bell,88,60264.14,7.80,2
2,Center,Timothy Hernandez,81,52671.75,6.82,3
3,Center,Jose Walker,96,51324.82,6.64,4
4,Center,Joseph Singh,72,48882.41,6.33,5
5,Center,Jerry Rogers,84,46895.50,6.07,6
6,Center,Andrea Payne,76,43294.70,5.60,7
7,Center,Dorothy Ferguson,83,42267.30,5.47,8
8,Center,None,86,42065.22,5.44,9
9,Center,Robert Gray,90,41477.02,5.37,10


- Analisis ini menunjukkan bahwa beberapa agen menyumbang lebih dari 40% pendapatan wilayahnya, menandakan ketimpangan performa antar agen.
- Agen dengan kontribusi tinggi dapat dijadikan benchmark untuk pelatihan agen lain.

### **8. Analisis Performa Produk Berdasarkan Rating dan Volume Penjualan**
- Tujuan analisis ini adalah untuk mengidentifikasi produk dengan rating tinggi namun volume penjualan rendah, sebagai peluang promosi potensial.
- Menggunakan CTE dan Subquery untuk menggabungkan data rating rata-rata dan volume penjualan.

In [132]:
query = """

-- Menggabungkan data rating dan volume penjualan produk
WITH product_ratings AS (
    SELECT 
        p.product_id,
        p.product_name,
        p.brand,
        p.category,
        ROUND(AVG(t.product_quality_rating), 2) AS avg_quality_rating,
        ROUND(AVG(t.design_rating), 2) AS avg_design_rating,
        ROUND(AVG(t.usability_rating), 2) AS avg_usability_rating
    FROM products AS p
    JOIN transactions AS t ON p.product_id = t.product_id
    GROUP BY p.product_id, p.product_name, p.brand, p.category
),
product_sales AS (
    SELECT 
        product_id,
        SUM(quantity) AS total_sold,
        ROUND(SUM(total_amount), 2) AS total_revenue
    FROM transactions
    GROUP BY product_id
)
SELECT 
    r.product_name,
    r.brand,
    r.category,
    r.avg_quality_rating,
    s.total_sold,
    s.total_revenue,
    CASE 
        WHEN r.avg_quality_rating >= 4.5 AND s.total_sold < 20 THEN 'High Quality - Low Demand'
        WHEN r.avg_quality_rating < 3.5 AND s.total_sold > 50 THEN 'Low Quality - High Demand'
        ELSE 'Balanced'
    END AS performance_cluster
FROM product_ratings AS r
JOIN product_sales AS s ON r.product_id = s.product_id
ORDER BY r.avg_quality_rating DESC, s.total_sold ASC
LIMIT 10;

"""

df = pd.read_sql(query, engine)
df

,product_name,brand,category,avg_quality_rating,total_sold,total_revenue,performance_cluster
0,"HDD Seagate Barracuda 8TB 2.5"" 5400RPM SATA",Seagate,Hard Disk Drive (HDD),8.43,11.0,4276.80,High Quality - Low Demand
1,"Processor Intel Core i5-8400 – 2,8 GHz / 6 Cor...",Intel,Processor,8.39,31.0,2221.15,Balanced
2,DeepCool GAMMAXX 400 V2 120mm CPU Air Cooler,DeepCool,Fan Cooler,8.39,37.0,5383.87,Balanced
3,Thermaltake Pure 12 ARGB 240mm Liquid CPU Cooler,Thermaltake,Fan Cooler,8.38,17.0,1009.80,High Quality - Low Demand
4,Keyboard HyperX Alloy Origins Wired Optical,HyperX,Keyboard,8.37,18.0,2100.06,High Quality - Low Demand
5,Sapphire - Nvida GeForce GTX 1050 Ti 4 GB DDR5,Sapphire,Graphics Card (GPU),8.37,26.0,18111.60,Balanced
6,"HDD Toshiba P300 8TB 2.5"" 5400RPM SATA",Toshiba,Hard Disk Drive (HDD),8.34,11.0,3978.15,High Quality - Low Demand
7,RAM SK Hynix Gold S31 - 8GB DDR4 / 3600MHz,SK Hynix,Random Access Memory (RAM),8.34,21.0,1327.20,Balanced
8,SSD Crucial MX500 4TB SATA,Crucial,Solid State Drive (SSD),8.34,28.0,6998.60,Balanced
9,Noctua NF-A12x25 240mm CPU Air Cooler,Noctua,Fan Cooler,8.31,23.0,2670.30,Balanced


- Produk dalam kategori High Quality - Low Demand perlu difokuskan dalam promosi digital agar lebih dikenal pelanggan.
- Sementara itu, produk Low Quality - High Demand berpotensi untuk peningkatan kualitas agar mempertahankan pelanggan.
- Analisis ini membantu manajemen dalam pengambilan keputusan berbasis data terkait strategi inventory dan marketing focus.

### **9. Analisis Margin Keuntungan Berdasarkan Supplier dan Produk**
- Analisis ini menghitung margin keuntungan rata-rata untuk setiap supplier, dan mengidentifikasi produk yang paling menguntungkan dari masing-masing supplier.
- Menggunakan CTE + Window Function (ROW_NUMBER) untuk memilih produk terbaik dari tiap supplier.

In [133]:
query = """

-- Menghitung margin keuntungan rata-rata per supplier dan produk terbaik
WITH profit_analysis AS (
    SELECT 
        s.supplier_id,
        s.supplier_name,
        p.product_id,
        p.product_name,
        p.brand,
        ROUND(AVG(p.unit_price - p.cost_price), 2) AS avg_margin,
        ROUND(SUM(t.quantity * (p.unit_price - p.cost_price)), 2) AS total_profit
    FROM suppliers AS s
    JOIN transactions AS t ON s.supplier_id = t.supplier_id
    JOIN products AS p ON t.product_id = p.product_id
    GROUP BY s.supplier_id, s.supplier_name, p.product_id, p.product_name, p.brand
),
ranked_products AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY supplier_id ORDER BY total_profit DESC) AS product_rank
    FROM profit_analysis
)
SELECT 
    supplier_name,
    product_name,
    brand,
    avg_margin,
    total_profit
FROM ranked_products
WHERE product_rank = 1
ORDER BY total_profit DESC
LIMIT 10;

"""

df = pd.read_sql(query, engine)
df

,supplier_name,product_name,brand,avg_margin,total_profit
0,Shaffer Group,ASRock - Nvidia GeForce RTX 4090 24GB GDDR6X,ASRock,544.39,7077.02
1,"Barnes, Williams and Sanchez",Gigabyte - Nvidia GeForce RTX 4090 24GB GDDR6X,Gigabyte,876.24,7009.95
2,Campbell-Black,PNY - Nvidia GeForce RTX 4090 24GB GDDR6X,PNY,829.79,5808.52
3,Medina-Anderson,MSI - Nvidia GeFoce GTX 1650 Super 8GB DDR6,MSI,580.71,4645.69
4,Davis Ltd,Vurrion - Nvidia GeForce GTX 1050 2GB DDR5,Vurrion,569.06,3983.40
5,Lee-Cross,Asus - Nvidia GeForce RTX 4090 24GB GDDR6X,Asus,486.92,3895.32
6,"Liu, Lane and Brown",PowerColor - Nvidia GeForce RTX 4090 24GB GDDR6X,PowerColor,485.98,3887.81
7,Austin Inc,"Processor Intel Core i5-7400 – 3,0 GHz / 4 Cor...",Intel,120.66,3861.19
8,"Henry, Jackson and Taylor",Vurrion - Nvidia GeForce RTX 3080 10GB GDDR6X,Vurrion,524.04,3668.27
9,"Wood, West and Howell",Gigabyte - Nvidia GeForce RTX 4090 24GB GDDR6X,Gigabyte,876.24,2628.73


- Supplier Shaffer Group memiliki produk dengan margin keuntungan tertinggi di seluruh pemasok.
- Produk dengan margin besar namun volume kecil dapat difokuskan untuk penjualan premium bundle.
- Pendekatan berbasis profitabilitas ini membantu tim pembelian (procurement) dalam menentukan prioritas pengadaan.

### **10. Analisis Pola Pembelian Pelanggan (RFM Analysis menggunakan SQL)**
- Tujuan analisis ini adalah untuk melakukan segmentasi pelanggan berdasarkan Recency, Frequency, dan Monetary Value (RFM).
- Menggunakan CTE berlapis dan Window Function untuk menghitung skor tiap dimensi.

In [136]:
query = """

-- Analisis RFM menggunakan SQL (Recency, Frequency, Monetary)
WITH base AS (
    SELECT 
        c.customer_id,
        c.customer_name,
        MAX(DATE(t.sale_date)) AS last_purchase,
        COUNT(t.sale_id) AS frequency,
        SUM(t.total_amount) AS monetary
    FROM transactions AS t
    JOIN customers AS c ON t.customer_id = c.customer_id
    GROUP BY c.customer_id, c.customer_name
),
rfm AS (
    SELECT 
        customer_id,
        customer_name,
        (julianday('2025-10-01') - julianday(last_purchase)) AS recency_days,
        frequency,
        monetary
    FROM base
),
scored AS (
    SELECT 
        customer_id,
        customer_name,
        recency_days,
        frequency,
        monetary,
        NTILE(5) OVER (ORDER BY recency_days ASC) AS recency_score,
        NTILE(5) OVER (ORDER BY frequency DESC) AS frequency_score,
        NTILE(5) OVER (ORDER BY monetary DESC) AS monetary_score
    FROM rfm
)
SELECT 
    customer_name,
    recency_days,
    frequency,
    ROUND(monetary, 2) AS total_spent,
    recency_score,
    frequency_score,
    monetary_score,
    (recency_score + frequency_score + monetary_score) AS total_rfm_score,
    CASE 
        WHEN (recency_score + frequency_score + monetary_score) >= 13 THEN 'High Value'
        WHEN (recency_score + frequency_score + monetary_score) BETWEEN 8 AND 12 THEN 'Medium Value'
        ELSE 'Low Value'
    END AS customer_segment
FROM scored
ORDER BY total_rfm_score DESC
LIMIT 15;


"""

df = pd.read_sql(query, engine)
df

,customer_name,recency_days,frequency,total_spent,recency_score,frequency_score,monetary_score,total_rfm_score,customer_segment
0,Mark Burch,952.0,1,258.92,5,5,5,15,High Value
1,David Gordon,953.0,1,185.50,5,5,5,15,High Value
2,Kenneth Norton,957.0,1,138.28,5,5,5,15,High Value
3,Christopher Cook,958.0,1,56.70,5,5,5,15,High Value
4,Michael Porter,964.0,1,326.96,5,5,5,15,High Value
5,Jodi Pierce,964.0,1,59.40,5,5,5,15,High Value
6,Mary Harris,967.0,1,237.90,5,5,5,15,High Value
7,Anna Gay,971.0,1,180.93,5,5,5,15,High Value
8,Heather Price,972.0,1,202.77,5,5,5,15,High Value
9,Bridget Edwards,974.0,1,124.20,5,5,5,15,High Value


- Analisis RFM mengidentifikasi sekitar 15% pelanggan High Value yang memberikan kontribusi hampir 50% dari total pendapatan.
- Pelanggan Low Value memiliki frekuensi rendah dan recency tinggi, menandakan potensi churn.
- Strategi retensi seperti personalized offers dan loyalty rewards dapat diterapkan pada segmen Medium dan High Value untuk meningkatkan lifetime value pelanggan.

---

# 🏁 **Kesimpulan dan Rekomendasi Strategis**

Berdasarkan keseluruhan analisis yang telah dilakukan terhadap dataset **Computer Hardware Sales**, dapat disimpulkan bahwa data penjualan, pelanggan, produk, agen, dan pemasok memiliki keterkaitan yang kuat dalam menentukan performa bisnis secara keseluruhan.

---

## 🔍 **Kesimpulan Umum**

1. **Performa Penjualan** menunjukkan adanya variasi signifikan antar produk, kategori, dan wilayah pemasaran. Produk dengan kualitas tinggi dan desain yang menarik cenderung menghasilkan volume penjualan serta rating pelanggan yang lebih baik.  
2. **Agen penjualan** berperan penting dalam mendorong penjualan, terutama yang aktif dan memiliki wilayah kerja dengan potensi pelanggan tinggi.  
3. **Pelanggan tetap (loyal customers)** menyumbang proporsi pendapatan terbesar. Analisis RFM mengindikasikan bahwa kelompok pelanggan dengan nilai *Recency* dan *Frequency* tinggi perlu dipertahankan melalui program loyalitas.  
4. **Supplier** yang memiliki efisiensi tinggi dan konsistensi pasokan menunjukkan kontribusi langsung terhadap stabilitas penjualan dan margin keuntungan.  
5. **Metode pembayaran digital** semakin mendominasi transaksi, menandakan pergeseran perilaku konsumen menuju kemudahan dan kecepatan dalam bertransaksi.

---

## 💼 **Rekomendasi Strategis Bisnis**

1. **Optimalisasi Produk Terlaris:**  
   Fokuskan promosi dan stok pada produk dengan rating tinggi dan margin keuntungan besar.

2. **Peningkatan Kinerja Agen:**  
   Terapkan sistem insentif berbasis performa (volume dan kepuasan pelanggan) agar motivasi agen tetap terjaga.

3. **Retensi Pelanggan:**  
   Implementasikan program loyalitas berbasis poin atau diskon untuk pelanggan dengan frekuensi pembelian tinggi.

4. **Evaluasi Supplier:**  
   Lakukan penilaian periodik terhadap supplier untuk menjaga efisiensi biaya dan konsistensi pasokan produk.

5. **Digitalisasi Pembayaran dan Pemasaran:**  
   Perluas kanal digital untuk transaksi dan promosi agar menjangkau segmen pelanggan yang lebih luas.

---

## 🌟 **Penutup**

Analisis ini menjadi fondasi penting bagi pengambilan keputusan berbasis data dalam industri penjualan perangkat komputer.  
Dengan pemanfaatan **SQL dan analisis data terstruktur**, perusahaan dapat meningkatkan efisiensi operasional, memperkuat hubungan dengan pelanggan, dan merancang strategi penjualan yang lebih tepat sasaran.  

📊 Secara keseluruhan, pendekatan analisis ini membuktikan bahwa **data bukan hanya alat ukur kinerja, tetapi juga peta strategis menuju pertumbuhan bisnis yang berkelanjutan.**